In [1]:
import os
import utils
import random
import dataset
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.contrib.eager as tfe

In [2]:
# configst
#tfe.enable_eager_execution()
sess=tf.Session()
%matplotlib inline

# load data
X=dataset.load_dataset("./ShapeNet",num_of_examples=2)
Y=None;

# net parameters
conv_filter_count = [96, 128, 256, 256, 256, 256]
deconv_filter_count = [128, 128, 128, 64, 32, 2]

[load_dataset] loading from ./ShapeNet
[construct_paths] parsing dir ./ShapeNet for ['.obj', '.mtl'] ...
... done
[write_renders_to_disk] writing renders to ./ShapeNet_Renders/02691156/28402efe2e3435d124fbfb66ea1f14f7 ... 
... done
[fetch_renders_from_disk] fetching renders from ./ShapeNet_Renders/02691156/28402efe2e3435d124fbfb66ea1f14f7 ... 
... done
[write_renders_to_disk] writing renders to ./ShapeNet_Renders/02691156/b7f6e94ddb5ecc2366cf1b4a8fc3914e ... 
... done
[fetch_renders_from_disk] fetching renders from ./ShapeNet_Renders/02691156/b7f6e94ddb5ecc2366cf1b4a8fc3914e ... 
... done


In [13]:
# encoder network
cur_tensor=tf.convert_to_tensor(X[0],dtype=tf.float32)
print(cur_tensor.shape)
k_s = [3,3]
for i in range(6): 
    ks=[7,7]if i is 0 else k_s  
    #with tf.name_scope("encoder_layer"):
    cur_tensor=tf.layers.conv2d(cur_tensor,filters=conv_filter_count[i],kernel_size= k_s)
    cur_tensor=tf.layers.max_pooling2d(cur_tensor,1,1)
    cur_tensor=tf.nn.relu(cur_tensor)
    print(cur_tensor.shape)
    
# flatten tensor before 3D-LSTM cells
cur_tensor=tf.contrib.layers.fully_connected(cur_tensor,1024)
print(cur_tensor.shape)

# RNN just 1 cell now
state=tf.random_normal([1,1024])
cell=tf.nn.rnn_cell.GRUCell(1024)
cell(cur_tensor,state)
print(cur_tensor.shape)

# recive input from neightboars via 3*3 conv
#hidden_state_1=cur_tensor,tf.random_uniform([3,3,3])
#print(cur_tensor.shape)


# collect hidden state from grid
hidden_states=tf.random_uniform([1,32,32,32,1024])
cur_tensor=hidden_states
print(cur_tensor.shape)

(10, 512, 512, 3)
(10, 510, 510, 96)
(10, 508, 508, 128)
(10, 506, 506, 256)
(10, 504, 504, 256)
(10, 502, 502, 256)
(10, 500, 500, 256)
(10, 500, 500, 1024)


ValueError: Input 0 of layer gru_cell_3 is incompatible with the layer: expected ndim=2, found ndim=4. Full shape received: [10, 500, 500, 1024]

In [ ]:
# decoder network
for i in range(6):
        cur_tensor=tf.(filters=deconv_filter_count[i],kernel_size=(3,3,3))(cur_tensor)
        cur_tensor=tf_keras.layers.UpSampling3D()(cur_tensor)
        cur_tensor=tf_keras.layers.Activation('relu')(cur_tensor)
        print(cur_tensor.shape)

In [ ]:
# serialize graph for tensorboard
writer = tf.summary.FileWriter("./logs/")
writer.add_graph(sess.graph)